## TODO
* Make "save_videos_dataset_as_frames" a DVC pipeline
* Extract patches from images
* Run classifiers on those patches
* Run mini-lstm on rgb-mean or something like that (should be enough to count the number of frames)
* Do augmentation
* Repeat the previous classifiers (they should fail)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import pandas as pd

from fight_classifier.data.video_to_images import (
    save_videos_dataset_as_frames)

project_root = Path("/home/matthieu/aviva/hiring-challenge")
frames_dir = project_root / 'dataset/raw_frames/'
videos_dir = project_root / 'dataset/Peliculas/'

videos_df = pd.read_csv(videos_dir / 'videos.csv')
videos_df

In [ ]:
# Save the videos to images
frames_df = save_videos_dataset_as_frames(
    videos_df=videos_df, videos_dir=videos_dir, frames_dir=frames_dir
)


In [ ]:
frames_df